# Day 2 PM - Practical Descriptive Statistics on SQL

You have already learnt basic SQL query syntaxes including DML, DDL, and DQL, especially today, we learn about group by, rank, and row_number. In this notebook, we will use our SQL knowledge to solve descriptive statistics problems. To narrowing our learning scope, we only use sqlite but in general, the concept and the syntax are not different so far. Furthermore, you can use other IDE such as Dbeaver, Datagrip, Navicat, etc.

In [ ]:
import sqlite3
import pandas as pd

## The Dataset
For this learning, we use data from Our World in Data about air pollution. You can access the github to look at this data or others: https://github.com/owid/owid-datasets/tree/master/datasets . You don't need download any data, we access the link directly using Pandas.

We have two csv files which are air pollutant emission from OECD and air pollutant death breakdown by age from IMHE. The description of each data delivered below:

1. **Air pollutant emission by OECD**\
Air pollutant emissions reported for OECD countries, where data is available, measured in tonnes per year.\
Indexed figures relate to changes since the year 1990 (1990 is assumed equal to 100). A figure lower than 100 indicates a reduction in emissions (e.g. 40 indicates a 60% reduction since 1990). Indexed figures are only available for countries with data extending to 1990.


2. **Air pollutant death breakdown by IMHE**\
Data relates to the number of deaths attributed to air pollution across age groups. This is measured based on attribution to all linked causes.\
IHME have developed a Comparative Risk Assessment (CRA) conceptual framework by which they have built a web of risk factors or causes which affect health outcomes.\
Such risk-outcome pairs (e.g. air pollution and lung cancer) are formed based on evidence links using methods such as cohort studies, randomised trials, and case-control studies. Once a risk-outcome pair has been formed, how does IHME begin to quantify the disease burden or number of deaths attributed to each risk? The CRA can be used for two different types of assessment: attributable burden and avoidable burden. 'Attributable burden' represents the reduction in current disease burden (or that of a given year) if population exposure had shifted to another counterfactual/hypothetical exposure level; 'avoidable burden' represents the potential future avoided burden if population exposure was to shift to a counterfactual level of exposure. Since the number of deaths is based on current or historical data, the data presented here is that of attributable burden.

In [ ]:
air_pollutant_emission = pd.read_csv('https://github.com/owid/owid-datasets/raw/master/datasets/Air%20Pollutant%20Emissions%20-%20OECD/Air%20Pollutant%20Emissions%20-%20OECD.csv')
air_pol_death_by_age = pd.read_csv('https://github.com/owid/owid-datasets/raw/master/datasets/Air%20pollution%20deaths%20breakdown%20by%20age%20-%20IHME/Air%20pollution%20deaths%20breakdown%20by%20age%20-%20IHME.csv')

In [ ]:
air_pollutant_emission.head()

,Entity,Year,Carbon Monoxide,NOx,Non-methane Volatile Organic Compounds (VOCs),PM₁₀,PM₂.₅,SO₂,Carbon Monoxide (Index),NOx (Index),Non-methane Volatile Organic Compounds (VOCs) (Index),PM₁₀ (Index),PM₂.₅ (Index),SO₂ (Index)
0,Australia,1990,5729,1621,1387.0,NaN,NaN,1555,100.00,100.00,100.00,NaN,NaN,100.00
1,Australia,1991,5747,1596,1373.0,NaN,NaN,1598,100.33,98.48,98.96,NaN,NaN,102.74
2,Australia,1992,5897,1642,1374.0,NaN,NaN,1707,102.93,101.31,99.08,NaN,NaN,109.76
3,Australia,1993,6122,1685,1390.0,NaN,NaN,1757,106.87,103.97,100.24,NaN,NaN,112.97
4,Australia,1994,6263,1687,1403.0,NaN,NaN,1819,109.33,104.08,101.12,NaN,NaN,116.97


General overview of air pollutant emission data. We see that there are some nulls in our data. let check them out that how much the nulls are in each column.

In [ ]:
air_pollutant_emission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 14 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Entity                                                 910 non-null    object 
 1   Year                                                   910 non-null    int64  
 2   Carbon Monoxide                                        910 non-null    int64  
 3   NOx                                                    910 non-null    int64  
 4   Non-methane Volatile Organic Compounds (VOCs)          908 non-null    float64
 5   PM₁₀                                                   636 non-null    float64
 6   PM₂.₅                                                  616 non-null    float64
 7   SO₂                                                    910 non-null    int64  
 8   Carbon Monoxide (Index)                           

The actual size of the data is 910 rows and seem that several columns severe the nulls. Since the data is numerical, we assume that the nulls can be imputed by 0, which mean no data. The aim of missing values handling in this hands on, we want to ease our work when we input this data into SQL table. We do not want to make our life difficult, don't we?

In [ ]:
air_pollutant_emission.fillna(0,inplace=True)
air_pollutant_emission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 910 entries, 0 to 909
Data columns (total 14 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Entity                                                 910 non-null    object 
 1   Year                                                   910 non-null    int64  
 2   Carbon Monoxide                                        910 non-null    int64  
 3   NOx                                                    910 non-null    int64  
 4   Non-methane Volatile Organic Compounds (VOCs)          910 non-null    float64
 5   PM₁₀                                                   910 non-null    float64
 6   PM₂.₅                                                  910 non-null    float64
 7   SO₂                                                    910 non-null    int64  
 8   Carbon Monoxide (Index)                           

Hurray! Our data is cleaned from missing value, at least there are no nulls anymore.

In [ ]:
air_pol_death_by_age.head()

,Entity,Year,Indoor (10 to 14 years),Indoor (15 to 19 years),Indoor (20 to 24 years),Indoor (25 to 29 years),Indoor (30 to 34 years),Indoor (35 to 39 years),Indoor (40 to 44 years),Indoor (45 to 49 years),...,Outdoor (45 to 49 years),Outdoor (5 to 9 years),Outdoor (50 to 54 years),Outdoor (55 to 59 years),Outdoor (60 to 64 years),Outdoor (65 to 69 years),Outdoor (70 to 74 years),Outdoor (75 to 79 years),Outdoor (80+ years),Outdoor (Under-5s)
0,Afghanistan,1990,78,45,43,235,335,523,830,1006,...,676,79,790,823,954,1045,890,576,440,5416
1,Afghanistan,1995,111,65,62,347,480,739,1141,1620,...,1068,108,1269,1331,1385,1477,1376,947,644,6067
2,Afghanistan,2000,125,76,73,421,592,895,1366,1858,...,1237,116,1637,1670,1695,1661,1583,1201,898,6062
3,Afghanistan,2005,184,93,90,511,702,1034,1520,2065,...,1332,206,1714,1960,1923,1873,1626,1236,1035,6771
4,Afghanistan,2006,183,92,89,517,714,1051,1539,2082,...,1364,204,1746,2007,2006,1931,1686,1274,1082,6511


We can see above that our second data seems okay, we have 36 columns which is a lot of work to do later. However, we have to check the data summary whether the nulls exist or not.

In [ ]:
air_pol_death_by_age.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1547 entries, 0 to 1546
Data columns (total 36 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Entity                    1547 non-null   object
 1   Year                      1547 non-null   int64 
 2   Indoor (10 to 14 years)   1547 non-null   int64 
 3   Indoor (15 to 19 years)   1547 non-null   int64 
 4   Indoor (20 to 24 years)   1547 non-null   int64 
 5   Indoor (25 to 29 years)   1547 non-null   int64 
 6   Indoor (30 to 34 years)   1547 non-null   int64 
 7   Indoor (35 to 39 years)   1547 non-null   int64 
 8   Indoor (40 to 44 years)   1547 non-null   int64 
 9   Indoor (45 to 49 years)   1547 non-null   int64 
 10  Indoor (5 to 9 years)     1547 non-null   int64 
 11  Indoor (50 to 54 years)   1547 non-null   int64 
 12  Indoor (55 to 59 years)   1547 non-null   int64 
 13  Indoor (60 to 64 years)   1547 non-null   int64 
 14  Indoor (65 to 69 years) 

There are no nulls in our sencod data and this is a relief. Finally, we will do some SQL stuffs with our SQL database.

### Creating and Connecting the SQLite Database

First of all, since we don't have a database, let's create sqlite database file named "day2pm.sqlite" or anything else that you want. But, before we do that, we need to define a function to create a connection to our sqlite database just like below:

In [ ]:
def create_connection(path):
    connection=sqlite3.connect(path)
    return(connection)

connection = create_connection("day2pm.sqlite")

After we create the database and connection, we need to define a function to send our query into the database. This function used for table creation and data insertion process. 

In [ ]:
def execute_query(connection, query):
    cursor = connection.cursor()
    cursor.execute(query)
    connection.commit()


### Creating Tables

Next, we define the DDL query to create the tables. We have two data so we have to make two tables which are air_pollutant_emission and air_pollutant_death_by_age in seperate query. Please adjust the column name with the pandas dataframes columns. You can edit the columns name as comfortable as you. Be carefull of the data type, check the data before you set the data type on the queries. After defining the queries for each table, we call the execute function and input the connection and query as the arguments for creating the tables.

In [ ]:
# Create air_pollutant_emission table query

create_table_1 = ''' 
CREATE TABLE IF NOT EXISTS air_pollutant_emission (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Entity TEXT NOT NULL,
    Year INTEGER,
    CO FLOAT,
    NOx FLOAT,
    Non_Methane_VOCs FLOAT,
    PM1_0 FLOAT,
    PM2_5 FLOAT,
    SO2 FLOAT,
    CO_index FLOAT,
    NOx_index FLOAT,
    Non_Methane_VOCs_index FLOAT,
    PM1_0_index FLOAT,
    PM2_5_index FLOAT,
    SO2_index FLOAT
)
'''

# Create air_pollutant_death_by_age table query

create_table_2 = '''
CREATE TABLE IF NOT EXISTS air_pollutant_death_by_age (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Entity TEXT NOT NULL,
    Year INTEGER,
    Indoor_Under_5 INTEGER,
    Indoor_5_to_9 INTEGER,
    Indoor_10_to_14 INTEGER,
    Indoor_15_to_19 INTEGER,
    Indoor_20_to_24 INTEGER,
    Indoor_25_to_29 INTEGER,
    Indoor_30_to_34 INTEGER,
    Indoor_35_to_39 INTEGER,
    Indoor_40_to_44 INTEGER,
    Indoor_45_to_49 INTEGER,
    Indoor_50_to_54 INTEGER,
    Indoor_55_to_59 INTEGER,
    Indoor_60_to_64 INTEGER,
    Indoor_65_to_69 INTEGER,
    Indoor_70_to_74 INTEGER,
    Indoor_75_to_79 INTEGER,
    Indoor_80_plus INTEGER,
    Outdoor_Under_5 INTEGER,
    Outdoor_5_to_9 INTEGER,
    Outdoor_10_to_14 INTEGER,
    Outdoor_15_to_19 INTEGER,
    Outdoor_20_to_24 INTEGER,
    Outdoor_25_to_29 INTEGER,
    Outdoor_30_to_34 INTEGER,
    Outdoor_35_to_39 INTEGER,
    Outdoor_40_to_44 INTEGER,
    Outdoor_45_to_49 INTEGER,
    Outdoor_50_to_54 INTEGER,
    Outdoor_55_to_59 INTEGER,
    Outdoor_60_to_64 INTEGER,
    Outdoor_65_to_69 INTEGER,
    Outdoor_70_to_74 INTEGER,
    Outdoor_75_to_79 INTEGER,
    Outdoor_80_plus INTEGER
)
'''

execute_query(connection, create_table_1)
execute_query(connection, create_table_2)

### Inserting the Data

In previous step, we only created the tables without inputing the data. In this step, we need to insert the data from the dataframes into the database tables. This is a lot of work actually so please be patient and careful. We use for loop to access each row on each dataframes and insert it to the data one by one.

In [ ]:
for i in range(len(air_pollutant_emission)):
    dat = air_pollutant_emission.values[i,:]
    insert_values_into_table_1 = f'''
    INSERT INTO air_pollutant_emission (
        Entity, Year, CO, NOx, Non_Methane_VOCs, PM1_0, PM2_5, SO2, CO_index, NOx_index, Non_Methane_VOCs_index, PM1_0_index, PM2_5_index, SO2_index)
    VALUES ("{dat[0]}", {dat[1]}, {dat[2]}, {dat[3]}, {dat[4]}, {dat[5]}, {dat[6]}, {dat[7]}, {dat[8]}, {dat[9]}, {dat[10]}, {dat[11]}, {dat[12]}, {dat[13]})
    '''
    execute_query(connection, insert_values_into_table_1)

In [ ]:
for i in range(len(air_pol_death_by_age)):
    dat = air_pol_death_by_age.values[i,:]
    insert_values_into_table_2 = f'''
    INSERT INTO air_pollutant_death_by_age (
        Entity, Year, Indoor_Under_5, Indoor_5_to_9, Indoor_10_to_14, Indoor_15_to_19, Indoor_20_to_24, Indoor_25_to_29, Indoor_30_to_34, Indoor_35_to_39,
        Indoor_40_to_44, Indoor_45_to_49, Indoor_50_to_54, Indoor_55_to_59, Indoor_60_to_64, Indoor_65_to_69, Indoor_70_to_74, Indoor_75_to_79, Indoor_80_plus,
        Outdoor_Under_5, Outdoor_5_to_9, Outdoor_10_to_14, Outdoor_15_to_19, Outdoor_20_to_24, Outdoor_25_to_29, Outdoor_30_to_34, Outdoor_35_to_39,
        Outdoor_40_to_44, Outdoor_45_to_49, Outdoor_50_to_54, Outdoor_55_to_59, Outdoor_60_to_64, Outdoor_65_to_69, Outdoor_70_to_74, Outdoor_75_to_79, Outdoor_80_plus)
    
    VALUES ("{dat[0]}", {dat[1]}, {dat[18]}, {dat[10]}, {dat[2]}, {dat[3]}, {dat[4]}, {dat[5]}, {dat[6]}, {dat[7]}, {dat[8]}, {dat[9]}, {dat[11]},
            {dat[12]}, {dat[13]}, {dat[14]}, {dat[15]}, {dat[16]}, {dat[17]}, {dat[35]}, {dat[27]}, {dat[19]}, {dat[20]}, {dat[21]}, {dat[22]},
            {dat[23]}, {dat[24]}, {dat[25]}, {dat[26]}, {dat[28]}, {dat[29]}, {dat[30]}, {dat[31]}, {dat[32]}, {dat[33]}, {dat[34]})
    '''
    execute_query(connection, insert_values_into_table_2)

Yeay, we have already transfer our dataframes into sqlite database successfully. To check our data or tables, please run the cell below:

In [ ]:
cursor = connection.cursor()

cursor.execute("SELECT name FROM sqlite_sequence;")
print(cursor.fetchall())

[('air_pollutant_emission',), ('air_pollutant_death_by_age',)]


Our tables are successfully created and now we check the data of each table. We will ask Pandas for help.

In [ ]:
pd.read_sql_query('select * from air_pollutant_emission limit 5',connection)

,id,Entity,Year,CO,NOx,Non_Methane_VOCs,PM1_0,PM2_5,SO2,CO_index,NOx_index,Non_Methane_VOCs_index,PM1_0_index,PM2_5_index,SO2_index
0,1,Australia,1990,5729.0,1621.0,1387.0,0.0,0.0,1555.0,100.00,100.00,100.00,0.0,0.0,100.00
1,2,Australia,1991,5747.0,1596.0,1373.0,0.0,0.0,1598.0,100.33,98.48,98.96,0.0,0.0,102.74
2,3,Australia,1992,5897.0,1642.0,1374.0,0.0,0.0,1707.0,102.93,101.31,99.08,0.0,0.0,109.76
3,4,Australia,1993,6122.0,1685.0,1390.0,0.0,0.0,1757.0,106.87,103.97,100.24,0.0,0.0,112.97
4,5,Australia,1994,6263.0,1687.0,1403.0,0.0,0.0,1819.0,109.33,104.08,101.12,0.0,0.0,116.97


In [ ]:
pd.read_sql_query('select * from air_pollutant_death_by_age limit 5',connection)

,id,Entity,Year,Indoor_Under_5,Indoor_5_to_9,Indoor_10_to_14,Indoor_15_to_19,Indoor_20_to_24,Indoor_25_to_29,Indoor_30_to_34,...,Outdoor_35_to_39,Outdoor_40_to_44,Outdoor_45_to_49,Outdoor_50_to_54,Outdoor_55_to_59,Outdoor_60_to_64,Outdoor_65_to_69,Outdoor_70_to_74,Outdoor_75_to_79,Outdoor_80_plus
0,1,Afghanistan,1990,8683,129,78,45,43,235,335,...,355,557,676,790,823,954,1045,890,576,440
1,2,Afghanistan,1995,10062,182,111,65,62,347,480,...,492,754,1068,1269,1331,1385,1477,1376,947,644
2,3,Afghanistan,2000,9980,194,125,76,73,421,592,...,599,907,1237,1637,1670,1695,1661,1583,1201,898
3,4,Afghanistan,2005,12263,381,184,93,90,511,702,...,667,977,1332,1714,1960,1923,1873,1626,1236,1035
4,5,Afghanistan,2006,11593,373,183,92,89,517,714,...,688,1004,1364,1746,2007,2006,1931,1686,1274,1082


Bravo! We did great.

## Practical Descriptive Statistics Problems

Minister of Environment and Foresty of The Republic of Indonesia, Mrs. Siti Nurbaya Bakar assigns a Data Scientist in her ministry which is YOU to give her some insights about the air pollution condition in OECD countries. To make your works easier, She has some questions that you have to based on data that stored in the sqlite database. The questions are:

1. Which OECD country that has the 16th-lowest in 1990 on CO level? Could you provide the data?
2. If we compare to the question number 1, which OECD country that has the 16th-lowest in 2005? is the answer still the same? You don't need give the data, just the country name. Give me a name.
3. Please give me a table contains the SO2 level in 1990 and the last of year record of Poland. Did it increase or decrease?
4. Which OECD country(s) that the PM1.0 level is in maximum value before 2000?
5. Could you please provide the death breakdown data by age of the top 2 of highest NOx level of OECD countries for Indoor Under 5 by mean, max, and min?
6. Which pollutant that has highest correlation to the Outdoor death breakdown by age under 30? (Consider the OECD countries only)

### Question 1

**Which OECD country that has the 16th-lowest in 1990 on CO level? Could you provide the data?**

In [ ]:
q1_a = '''
select Entity, Year, CO,
        rank() over(
            order by CO asc
           ) as rank
from air_pollutant_emission
where Entity != 'OECD - Total' and Entity != 'Europe' and Year=1990
group by Entity, Year, CO;
'''
pd.read_sql_query(q1_a,connection)

,Entity,Year,CO,rank
0,Iceland,1990,59.0,1
1,Estonia,1990,215.0,2
2,Slovenia,1990,324.0,3
3,Ireland,1990,348.0,4
4,Latvia,1990,386.0,5
5,Luxembourg,1990,463.0,6
6,Slovakia,1990,505.0,7
7,New Zealand,1990,598.0,8
8,Finland,1990,709.0,9
9,Denmark,1990,741.0,10


We can see that if we use rank() and use order by CO, Greece and The Netherlands have the same rank since they have the same CO level in 1990. So, the answer is both Greece and The Netherlands. Different with rank(), row_number() will give ordered number based on the order and it does not care about the value. So the result is:

In [ ]:
q1_b = '''
select Entity, Year, CO,
        row_number() over(
            order by CO asc
           ) as rank
       from air_pollutant_emission
where Entity != 'OECD - Total' and Entity != 'Europe' and Year=1990
group by Entity, Year, CO;
'''
pd.read_sql_query(q1_b,connection)

,Entity,Year,CO,rank
0,Iceland,1990,59.0,1
1,Estonia,1990,215.0,2
2,Slovenia,1990,324.0,3
3,Ireland,1990,348.0,4
4,Latvia,1990,386.0,5
5,Luxembourg,1990,463.0,6
6,Slovakia,1990,505.0,7
7,New Zealand,1990,598.0,8
8,Finland,1990,709.0,9
9,Denmark,1990,741.0,10


If we asked that which country that has the 16th-lowest CO level in 2016, we only have an answer which is Greece based on row_number() calculation.

### Question 2

**If we compare to the question number 1, which OECD country that has the 16th-lowest in 2005? is the answer still the same? You don't need give the data, just the country name. Give me a name.**

Since Mrs. Minister want a name only, we use row_number instead and we do not need Pandas since we do not show the table

In [ ]:
q2 = '''
select Entity from (

    select Entity,
        row_number() over(
            order by CO asc
           ) as rank
    from air_pollutant_emission as ape
    where Entity != 'OECD - Total' and Entity != 'Europe' and Year=2005
    group by Entity, Year, CO)

where rank=16
;
'''
cursor = connection.cursor()
cursor.execute(q2)
print('The 16th-lowest CO level in 2005:',cursor.fetchall()[0][0])

The 16th-lowest CO level in 2005: Czech Republic


Greece is not longer be the 16th-lowest CO Level in 2005, but Czech Republic instead.

### Question 3

**Please give me a table contains the SO2 level in 1990 and the last of year record of Poland. Did it increase or decrease?**

In [ ]:
q3 = '''
select Entity, Year, SO2 from air_pollutant_emission as ape
where Entity = 'Poland' and (Year = 1990 or Year = (select max(Year) from air_pollutant_emission where Entity='Poland' ))
group by Entity, SO2
order by Year
'''
pd.read_sql_query(q3,connection)

,Entity,Year,SO2
0,Poland,1990,3210.0
1,Poland,2015,690.0


In 2015, SO2 emission level of Poland is decreased from 1990

### Question 4

**Which OECD country(s) that the PM1.0 level is in maximum value before 2000?**

In [ ]:
q4='''
select * from (select Entity, Year, max(PM1_0) as "Max PM1.0" from air_pollutant_emission as ape
where Entity!="Europe" and Entity!="OECD - Total" group by Entity)
where Year < 2000
'''
pd.read_sql_query(q4,connection)

,Entity,Year,Max PM1.0
0,Australia,1990,0.0
1,Austria,1990,40.0
2,France,1991,613.0
3,Germany,1995,329.0
4,Greece,1990,0.0
5,Ireland,1990,43.0
6,Israel,1996,0.0
7,Italy,1991,278.0
8,Japan,1990,0.0
9,Luxembourg,1990,17.0


### Question 5

**Could you please provide the death breakdown data by age of the top 2 of highest NOx level of OECD countries in 2015 for Indoor Under 5 by mean, maximum, and minimum?**

In [ ]:
q5 = '''
select dat.Entity, avg(apda.Indoor_Under_5) as mean,
       max(apda.Indoor_Under_5) as maximum,
       min(apda.Indoor_Under_5) as minimum
from (select Entity,NOx,rank() over(order by NOx desc) as rank
        from air_pollutant_emission
        where Entity!="Europe" and Entity!="OECD - Total" and Year=2015
        group by Entity, NOx limit 2) as dat
join air_pollutant_death_by_age as apda on dat.Entity = apda.Entity
group by dat.Entity;
'''

pd.read_sql_query(q5,connection)

,Entity,mean,maximum,minimum
0,Australia,0.000000,0,0
1,United States,1.285714,2,1
